In [47]:
import pandas as pd


def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()


def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

import talib
import numpy as np

close = np.random.random(100)
print(close)
sma = talib.SMA(close) # deze functie neem een array en geeft een array terug, waarbij len(input) == len(output)
print(sma)
rsi = talib.RSI(close, 30)
print(rsi)

[0.77345358 0.97781161 0.89817204 0.16908204 0.21189806 0.2579852
 0.10627747 0.63176418 0.02915486 0.11183581 0.22341536 0.86380043
 0.36138552 0.64334258 0.0061691  0.27776856 0.95597233 0.23464584
 0.73085651 0.58148902 0.47697931 0.06290371 0.50457212 0.79668481
 0.80905841 0.84432797 0.05467827 0.43023135 0.5988665  0.61884159
 0.03887354 0.96058868 0.19753824 0.86388988 0.63754835 0.12328089
 0.3149831  0.22966954 0.75893909 0.373324   0.43720229 0.63359915
 0.82769907 0.93520729 0.35639289 0.12391031 0.33784639 0.7009607
 0.50793544 0.55860064 0.35761056 0.54964505 0.78150453 0.81492509
 0.02027197 0.80956814 0.3270082  0.49019832 0.88081766 0.24808095
 0.52157239 0.80595643 0.41985858 0.1301054  0.41311211 0.55438923
 0.82970925 0.2293921  0.86355514 0.82415262 0.21369906 0.99227898
 0.71253131 0.51818844 0.95322762 0.50408602 0.64603871 0.57182593
 0.20693254 0.55173849 0.10072691 0.98344449 0.32006051 0.82383634
 0.87056962 0.94318452 0.95691128 0.95138295 0.68788864 0.390310

In [19]:
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        # De eerste is de functie, de overige argumenten zijn argumenten voor die functie.
        # self.data is de dataframe. Hoe kan je deze los meegeven zonder
        # https://github.com/kernc/backtesting.py/blob/master/backtesting/backtesting.py#L78
        self.ma10 = self.I(talib.SMA, self.data.Close, 10)
        self.ma20 = self.I(talib.SMA, self.data.Close, 20)
        self.ma50 = self.I(talib.SMA, self.data.Close, 50)
        self.ma100 = self.I(talib.SMA, self.data.Close, 100)
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(talib.RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', talib.RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

In [20]:
from backtesting.test import GOOG

print(GOOG.head())
backtest = Backtest(GOOG, System, commission=.002)
backtest.run()



              Open    High     Low   Close    Volume
2004-08-19  100.00  104.06   95.96  100.34  22351900
2004-08-20  101.01  109.08  100.50  108.31  11428600
2004-08-23  110.75  113.48  109.05  109.40   9137200
2004-08-24  111.24  111.60  103.57  104.87   7631300
2004-08-25  104.96  108.00  103.88  106.00   4598900


Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    2.607076
Equity Final [$]                   9894.21654
Equity Peak [$]                   10834.10988
Return [%]                          -1.057835
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   -0.124687
Volatility (Ann.) [%]                4.496176
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -9.888707
Avg. Drawdown [%]                   -4.881069
Max. Drawdown Duration     2653 days 00:00:00
Avg. Drawdown Duration      564 days 00:00:00
# Trades                                    4
Win Rate [%]                             25.0
Best Trade [%]                       9.687579
Worst Trade [%]                     -4.456159
Avg. Trade [%]                    

In [21]:
backtest.optimize(d_rsi=range(10, 35, 5),
                  w_rsi=range(10, 35, 5),
                  level=range(30, 80, 10))

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   27.094972
Equity Final [$]                  23304.82018
Equity Peak [$]                   24136.96578
Return [%]                         133.048202
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   10.435363
Volatility (Ann.) [%]               14.731717
Sharpe Ratio                          0.70836
Sortino Ratio                        1.186656
Calmar Ratio                         0.606959
Max. Drawdown [%]                  -17.192867
Avg. Drawdown [%]                   -3.596989
Max. Drawdown Duration      693 days 00:00:00
Avg. Drawdown Duration       71 days 00:00:00
# Trades                                   30
Win Rate [%]                             60.0
Best Trade [%]                      25.034669
Worst Trade [%]                     -7.914445
Avg. Trade [%]                    

In [22]:
backtest.plot()

Row(id='2644', ...)

In [24]:
import tpqoa
oanda = tpqoa.tpqoa('oanda.cfg')
oanda.get_instruments()

[('AUD/CAD', 'AUD_CAD'),
 ('AUD/CHF', 'AUD_CHF'),
 ('AUD/HKD', 'AUD_HKD'),
 ('AUD/JPY', 'AUD_JPY'),
 ('AUD/NZD', 'AUD_NZD'),
 ('AUD/SGD', 'AUD_SGD'),
 ('AUD/USD', 'AUD_USD'),
 ('Australia 200', 'AU200_AUD'),
 ('Bitcoin', 'BTC_USD'),
 ('Bitcoin Cash', 'BCH_USD'),
 ('Brent Crude Oil', 'BCO_USD'),
 ('Bund', 'DE10YB_EUR'),
 ('CAD/CHF', 'CAD_CHF'),
 ('CAD/HKD', 'CAD_HKD'),
 ('CAD/JPY', 'CAD_JPY'),
 ('CAD/SGD', 'CAD_SGD'),
 ('CHF/HKD', 'CHF_HKD'),
 ('CHF/JPY', 'CHF_JPY'),
 ('CHF/ZAR', 'CHF_ZAR'),
 ('China A50', 'CN50_USD'),
 ('China H Shares', 'CHINAH_HKD'),
 ('Copper', 'XCU_USD'),
 ('Corn', 'CORN_USD'),
 ('EUR/AUD', 'EUR_AUD'),
 ('EUR/CAD', 'EUR_CAD'),
 ('EUR/CHF', 'EUR_CHF'),
 ('EUR/CZK', 'EUR_CZK'),
 ('EUR/DKK', 'EUR_DKK'),
 ('EUR/GBP', 'EUR_GBP'),
 ('EUR/HKD', 'EUR_HKD'),
 ('EUR/HUF', 'EUR_HUF'),
 ('EUR/JPY', 'EUR_JPY'),
 ('EUR/NOK', 'EUR_NOK'),
 ('EUR/NZD', 'EUR_NZD'),
 ('EUR/PLN', 'EUR_PLN'),
 ('EUR/SEK', 'EUR_SEK'),
 ('EUR/SGD', 'EUR_SGD'),
 ('EUR/TRY', 'EUR_TRY'),
 ('EUR/USD', 'EUR_U

In [60]:
data = oanda.get_history(instrument='EUR_USD',
                  start='2015-07-01',
                  end='2021-05-31',
                  granularity='D',
                  price='A')
data.index = pd.to_datetime(data.index) # format?
data.rename_axis(None, inplace=True)
data.rename(columns = {'o':'Open', 'h':'High', 'l':'Low', 'c':'Close', 'volume':'Volume'}, inplace = True)
data = data[data.complete].drop(columns=['complete'])
print(GOOG.head())
print(data.head())


              Open    High     Low   Close    Volume
2004-08-19  100.00  104.06   95.96  100.34  22351900
2004-08-20  101.01  109.08  100.50  108.31  11428600
2004-08-23  110.75  113.48  109.05  109.40   9137200
2004-08-24  111.24  111.60  103.57  104.87   7631300
2004-08-25  104.96  108.00  103.88  106.00   4598900
                        Open     High      Low    Close  Volume
2015-06-30 21:00:00  1.11431  1.11730  1.10439  1.10540   37708
2015-07-01 21:00:00  1.10540  1.11231  1.10333  1.10852   34264
2015-07-02 21:00:00  1.10847  1.11215  1.10662  1.11212   21333
2015-07-05 21:00:00  1.10000  1.10968  1.09717  1.10573   46873
2015-07-06 21:00:00  1.10574  1.10609  1.09171  1.10126   48877


In [61]:
def get_data(oanda, instrument='EUR_USD', start='2020-07-01',end='2021-05-31',granularity='D', price='A'):
        data = oanda.get_history(instrument=instrument, start=start, end=end, granularity=granularity, price=price)
        data.rename(columns = {'o':'Open', 'h':'High', 'l':'Low', 'c':'Close', 'volume':'Volume'}, inplace = True)
        data = data[data.complete].drop(columns=['complete'])
        # TODO: timeperiod, eventuele nans opvullen
             
             
             

In [62]:
atr = talib.ATR(data["High"], data["Low"], data["Close"], timeperiod=14)
print(atr)

2015-06-30 21:00:00         NaN
2015-07-01 21:00:00         NaN
2015-07-02 21:00:00         NaN
2015-07-05 21:00:00         NaN
2015-07-06 21:00:00         NaN
                         ...   
2021-05-24 21:00:00    0.006728
2021-05-25 21:00:00    0.006827
2021-05-26 21:00:00    0.006624
2021-05-27 21:00:00    0.006667
2021-05-30 21:00:00    0.006535
Length: 1536, dtype: float64


In [63]:
backtest = Backtest(data, System, commission=.002)
backtest.run()

Start                     2015-06-30 21:00:00
End                       2021-05-30 21:00:00
Duration                   2161 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]               10.626018
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [89]:
# https://github.com/tomyitav/talib-extensions/blob/master/TechnicalAnalysis/src/indicators/TalibExtension.py
from backtesting.lib import crossover
def TSI(close, slowPeriod=25, fastPeriod=13):
	if ((fastPeriod >= close.size) | (slowPeriod >= close.size)):
		return;
	momentumVector = np.zeros(close.size);
	momentumVector[1:] = close[1:] - close[0:(close.size-1)]
	absMomentumVector = np.abs(momentumVector);
	k1 = 2/(slowPeriod+1);
	k2 = 2/(fastPeriod+1);
	
	ema1 = np.zeros(close.size);
	ema2 = np.copy(ema1);
	ema3 = np.copy(ema1);
	ema4 = np.copy(ema1);
	
	for i in range(1,close.size):
		ema1[i] = k1 * (momentumVector[i]-ema1[i-1]) + ema1[i-1];
		ema2[i] = k2 * (ema1[i]-ema2[i-1])   + ema2[i-1];
		ema3[i] = k1 * (absMomentumVector[i]-ema3[i-1]) + ema3[i-1];
		ema4[i] = k2 * (ema3[i]-ema4[i-1])   + ema4[i-1];

	tsi = 100 * np.divide(ema2, ema4)
	return tsi;

class NNFX(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        # De eerste is de functie, de overige argumenten zijn argumenten voor die functie.
        # self.data is de dataframe. Hoe kan je deze los meegeven zonder
        # https://github.com/kernc/backtesting.py/blob/master/backtesting/backtesting.py#L78
        
        # bewaar deze even voor een simpele exit
        self.ma10 = self.I(talib.SMA, self.data.Close, 10)
        
        #talib.ATR(data["High"], data["Low"], data["Close"], timeperiod=14)
        self.base_atr = self.I(talib.ATR,self.data.High, self.data.Low, self.data.Close, timeperiod=14)
        
        self.c_tsi = self.I(TSI, self.data.Close)
        self.c_tsi_signal = self.I(talib.EMA, self.c_tsi, timeperiod=10)
        
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(talib.RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', talib.RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        low_atr = price - self.base_atr[-1]
        high_atr = price + self.base_atr[-1]
        tsi = self.c_tsi[-1]
        tsi_signal = self.c_tsi_signal[-1]
        
        # je mag nog geen position hebbem
        positioncheck = not self.position
        
        # check baseline
        baselinecheck =  low_atr < price < high_atr
        
        # check TSI crossover
        tsi_check = crossover(self.c_tsi, self.c_tsi_signal)
        
        print(price, baselinecheck, positioncheck, tsi_check)
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if positioncheck and baselinecheck and tsi_check:
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()
            
    def buy():

In [90]:
backtest = Backtest(data, NNFX, commission=.002)
backtest.run()

1.09209 True True True
1.11047 True False False
1.12114 True False False
1.11617 True False False
1.11941 True False False
1.12961 True False False
1.12958 True False False
1.13243 True False False
1.12596 True False False
1.11578 True False False
1.11449 True False False
1.11286 True False False
1.11087 True False False
1.11334 True False False
1.10306 True False False
1.10203 True False False
1.10148 True False False
1.10192 True False False
1.09367 True False False
1.0874 True False False
1.08699 True False False
1.08693 True False False
1.09579 True False False
1.10099 True False False
1.10148 True False False
1.10122 True False False
1.10006 True False False
1.1181 True False True
1.1152 True False False
1.11046 True False False
1.11097 True False False
1.12264 True False False
1.13194 True False False
1.12733 True False False
1.12421 True False False
1.1218 True False False
1.11821 True False False
1.11762 True False False
1.11723 True False False
1.11964 True False False
1.12937

/tmp/ipykernel_1317/417164584.py:23: RuntimeWarning: invalid value encountered in divide
  tsi = 100 * np.divide(ema2, ema4)


Start                     2015-06-30 21:00:00
End                       2021-05-30 21:00:00
Duration                   2161 days 00:00:00
Exposure Time [%]                   88.151042
Equity Final [$]                 10964.675404
Equity Peak [$]                  11383.050793
Return [%]                           9.646754
Buy & Hold Return [%]               10.626018
Return (Ann.) [%]                    2.212546
Volatility (Ann.) [%]                7.960602
Sharpe Ratio                         0.277937
Sortino Ratio                        0.409596
Calmar Ratio                         0.148191
Max. Drawdown [%]                  -14.930363
Avg. Drawdown [%]                   -2.153883
Max. Drawdown Duration     1215 days 00:00:00
Avg. Drawdown Duration      102 days 00:00:00
# Trades                                    4
Win Rate [%]                             50.0
Best Trade [%]                      10.354907
Worst Trade [%]                     -2.528158
Avg. Trade [%]                    